# Calculating diversity and mutation

## Calculating mutational load
To calculate mutational load, the functions from `immcantation` suite's `shazam` [[Gupta2015]](https://academic.oup.com/bioinformatics/article/31/20/3356/195677) can be accessed via `rpy2` to work with the `dandelion` class object.

This can be run immediately after `pp.reassign_alleles` during the reannotation pre-processing stage because the required germline columns should be present in the genotyped `.tsv` file. I would recommend to run this after TIgGER [[Gadala-Maria2015]](https://www.pnas.org/content/112/8/E862), after the v_calls were corrected. Otherwise, if the reannotation was skipped, you can run it now as follows:

<b>Import modules</b>

In [ ]:
import os
import dandelion as ddl

ddl.logging.print_header()

In [ ]:
# change directory to somewhere more workable
os.chdir(os.path.expanduser("~/Downloads/dandelion_tutorial/"))
# I'm importing scanpy here to make use of its logging module.
import scanpy as sc

sc.settings.verbosity = 3
import warnings

warnings.filterwarnings("ignore")
sc.logging.print_header()

<b>Read in the previously saved files</b>

In [ ]:
adata = sc.read_h5ad("adata.h5ad")
adata

In [ ]:
vdj = ddl.read_h5ddl("dandelion_results.h5ddl")
vdj

In [ ]:
# let's recreate the vdj object with only the first two samples
subset_data = vdj.data[
    vdj.data["sample_id"].isin(["sc5p_v2_hs_PBMC_1k", "sc5p_v2_hs_PBMC_10k"])
]
subset_data

In [ ]:
# create a new Dandelion class with this subset
vdj2 = ddl.Dandelion(subset_data)
vdj2

### `store_germline_reference`

We can store the corrected germline fasta files (after running TIgGER) in the `Dandelion` class as a dictionary.

In [ ]:
# update the germline using the corrected files after tigger
vdj2.store_germline_reference(
    corrected="tutorial_scgp1/tutorial_scgp1_heavy_igblast_db-pass_genotype.fasta",
    germline=None,
    org="human",
)

### `pp.create_germlines`

Then we run `pp.create_germline` to (re)create the `germline_alignment_d_mask` column in the data. This works by calling `CreateGermlines.py` with only `-d` and `-r` options. Add further arguments with `additional_args` like below for your needs. See https://changeo.readthedocs.io/en/stable/examples/germlines.html for more info.

In [ ]:
ddl.pp.create_germlines(vdj2, additional_args=["--vf", "v_call_genotyped"])

Ensure that the `germline_alignment_d_mask` column is populated or subsequent steps will fail.

In [ ]:
vdj2.data[["v_call_genotyped", "germline_alignment_d_mask"]]

The default behaviour is to mask the D region with Ns with option.

### `pp.quantify_mutations`

The options for `pp.quantify_mutations` are the same as the basic mutational load analysis [vignette](https://shazam.readthedocs.io/en/version-0.1.8---mutation-profiling-enhancements/vignettes/Mutation-Vignette/) [[Gupta2015]](https://academic.oup.com/bioinformatics/article/31/20/3356/195677). The default behavior is to sum all mutations scores (heavy and light chains, silent and replacement mutations) for the same cell.

Again, this function can be run immediately after `pp.reassign_alleles` on the genotyped `.tsv` files (without loading into `pandas` or `Dandelion`). Here I'm illustrating a few other options that may be useful.

In [ ]:
# switching back to using the full vdj object
ddl.pp.quantify_mutations(vdj)

In [ ]:
ddl.pp.quantify_mutations(vdj, combine=False)

Specifying `split_locus = True` will split up the results for the different chains.

In [ ]:
ddl.pp.quantify_mutations(vdj, split_locus=True)

To update the `AnnData` object, simply rerun `tl.transfer`.

In [ ]:
ddl.tl.transfer(adata, vdj)

In [ ]:
adata

In [ ]:
from scanpy.plotting.palettes import default_28, default_102

sc.set_figure_params(figsize=[4, 4])
ddl.pl.clone_network(
    adata,
    color=[
        "clone_id",
        "mu_count",
        "mu_count_seq_r",
        "mu_count_seq_s",
        "mu_count_IGH",
        "mu_count_IGL",
    ],
    ncols=2,
    legend_loc="none",
    legend_fontoutline=3,
    edges_width=1,
    palette=default_28 + default_102,
    color_map="viridis",
    size=20,
)

## Calculating diversity

*Disclaimer: the functions here are experimental. Please look to other sources/methods for doing this properly. Also, would appreciate any help to help me finalise this!* 

`tl.clone_rarefaction` and `pl.clone_rarefaction`

We can use `pl.clone_rarefaction` to generate rarefaction curves for the clones. `tl.clone_rarefaction` will populate the `.uns` slot with the results. `groupby` option must be specified. In this case, I decided to group by sample. The function will only work on an `AnnData` object and not a `Dandelion` object.

In [ ]:
ddl.pl.clone_rarefaction(adata, color="sampleid")

### `ddl.tl.clone_diversity`

`tl.clone_diversity` allows for calculation of diversity measures such as <b>Chao1</b>, <b>Shannon Entropy</b> and <b>Gini indices</b>. 

While the function can work on both `AnnData` and `Dandelion` objects, the methods for gini index calculation will only work on a `Dandelion` object as it requires access to the network. 

For Gini indices, we provide several types of measures, inspired by bulk BCRseq analysis methods from [[Bashford-Rogers2013]](https://genome.cshlp.org/content/23/11/1874):

The following two indices are returned with `metric="clone_network"`.
   
   <b>I. network cluster/clone size Gini index</b> 
    
   In a contracted BCR network (where identical BCRs are collapsed into the same node/vertex), disparity in the distribution should be correlated to the amount of mutation events i.e. larger networks should indicate more mutation events and smaller networks should indicate lesser mutation events.

   <b>II. network vertex/node size Gini index</b>
    
   In the same contracted network, we can count the number of merged/contracted nodes; nodes with higher count numbers indicate more clonal expansion. Thus, disparity in the distribution of count numbers (referred to as vertex size) should be correlated to the overall clonality i.e. clones with larger vertex sizes are more monoclonal and clones with smaller vertex sizes are more polyclonal.
    
Therefore, a Gini index of 1 on either measures repesents perfect inequality (i.e. monoclonal and highly mutated) and a value of 0 represents perfect equality (i.e. polyclonal and unmutated).

<div class="alert alert-warning">

Note

However, there are a few limitations/challenges that comes with single-cell data: 

- <b>A.</b> In the process of contracting the network, we discard the single-cell level information. <br><br>

- <b>B.</b> Contraction of network is very slow, particularly when there is a lot of clonally-related cells. <br><br>

- <b>C.</b> For the full implementation and interpretation of both measures, although more evident with cluster/clone size, it requires the BCR repertoire to be reasonably/deeply sampled and we know that this is currently limited by the low recovery from single cell data with current technologies.

</div>

Therefore, we implement a few work around options, and 'experimental' options below, to try and circumvent these issues.

Firstly, as a work around for (C), the cluster size gini index can be calculated before or after network contraction. If performing before network contraction (default), it will be calculated based on the size of subgraphs of connected components in the main graph. This will retain the single-cell information and should appropriately show the distribution of the data. If performing after network contraction, the calculation is performed after network contraction, achieving the same effect as the method for bulk BCR-seq as described above. This option can be toggled by `use_contracted` and only applies to network cluster size gini index calculation.

   <b>III. clone centrality Gini index</b> - `metric="clone_centrality"`
   
   Node/vertex closeness centrality indicates how tightly packed clones are (more clonally related) and thus the distribution of the number of cells connected in each clone informs on whether clones in general are more monoclonal or polyclonal.

   <b>IV. clone degree Gini index</b> - `metric="clone_degree"`
   
   Node/vertex degree indicates how many cells are connected to an individual cell, another indication of how clonally related cells are. However, this would also highlight cells that are in the middle of large networks but are not necessarily within clonally expanded regions (e.g. intermediate connecting cells within the minimum spanning tree).
 
   <b>V. clone size Gini index</b> - `metric="clone_size"`
   
   This is not to be confused with the network cluster size gini index calculation above as this doesn't rely on the network, although the values should be similar. This is just a simple implementation based on the data frame for the relevant `clone_id` column. By default, this metric is also returned when running `metric=clone_centrality` or `metric=clone_degree`.

<div class="alert alert-warning">

Note

For (I) and (II), we can specify `expanded_only` option to compute the statistic for all clones or expanded only clones. Unlike options (I) and (II), the current calculation for (III) and (IV) is largely influenced by the amount of expanded clones i.e. clones with at least 2 cells, and not affected by the number of singleton clones because singleton clones will have a value of 0 regardless.
</div>

The diversity functions also have the option to perform downsampling to a fixed number of cells, or to the smallest sample size specified via `groupby` (default) so that sample sizes are even when comparing between groups.

if `return_table=True`, a data frame is returned; otherwise, the value gets added to the `AnnData.obs` or `Dandelion.metadata` accordingly.

In [ ]:
sc.settings.verbosity = 1  # it gets very noisy
ddl.tl.clone_diversity(
    vdj, groupby="sample_id", method="gini", metric="clone_network"
)
ddl.tl.clone_diversity(
    vdj, groupby="sample_id", method="gini", metric="clone_centrality"
)
ddl.tl.transfer(adata, vdj)

In [ ]:
ddl.pl.clone_network(
    adata,
    color=[
        "clone_network_cluster_size_gini",
        "clone_network_vertex_size_gini",
        "clone_size_gini",
        "clone_centrality_gini",
    ],
    ncols=2,
    size=20,
)

With these particular samples, because there is not many expanded clones in general, the gini indices are quite low when calculated within each sample. We can re-run it by specifying `expanded_only = True` to only factor in expanded clones. We also specify the `key_added` option to create a new column instead of writing over the original columns.

In [ ]:
ddl.tl.clone_diversity(
    vdj,
    groupby="sample_id",
    method="gini",
    metric="clone_network",
    expanded_only=True,
    key_added=[
        "clone_network_cluster_size_gini_expanded",
        "clone_network_vertex_size_gini_expanded",
    ],
)
ddl.tl.transfer(adata, vdj)

In [ ]:
ddl.pl.clone_network(
    adata,
    color=[
        "clone_network_cluster_size_gini_expanded",
        "clone_network_vertex_size_gini_expanded",
    ],
    ncols=2,
    size=20,
)

We can also choose not to update the metadata to return a pandas dataframe.

In [ ]:
gini = ddl.tl.clone_diversity(
    vdj, groupby="sample_id", method="gini", return_table=True
)
gini

In [ ]:
gini2 = ddl.tl.clone_diversity(
    vdj,
    groupby="sample_id",
    method="gini",
    return_table=True,
    expanded_only=True,
    key_added=[
        "clone_network_cluster_size_gini_expanded",
        "clone_network_vertex_size_gini_expanded",
    ],
)
gini2

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

p = sns.scatterplot(
    x="clone_network_cluster_size_gini",
    y="clone_network_vertex_size_gini",
    data=gini,
    hue=gini.index,
    palette=dict(
        zip(adata.obs["sampleid"].cat.categories, adata.uns["sampleid_colors"])
    ),
)
p.set(ylim=(-0.1, 1), xlim=(-0.1, 1))
plt.legend(bbox_to_anchor=(1, 0.5), loc="center left", frameon=False)
p

In [ ]:
p2 = sns.scatterplot(
    x="clone_network_cluster_size_gini_expanded",
    y="clone_network_vertex_size_gini_expanded",
    data=gini2,
    hue=gini2.index,
    palette=dict(
        zip(adata.obs["sampleid"].cat.categories, adata.uns["sampleid_colors"])
    ),
)
p2.set(ylim=(-0.1, 1), xlim=(-0.1, 1))
plt.legend(bbox_to_anchor=(1, 0.5), loc="center left", frameon=False)
p2

We can also visualise what the results for the clone centrality gini indices.

In [ ]:
gini = ddl.tl.clone_diversity(
    vdj,
    groupby="sample_id",
    method="gini",
    metric="clone_centrality",
    return_table=True,
)
gini

In [ ]:
# not a great example because there's only 1 big clone in 1 sample.
p = sns.scatterplot(
    x="clone_size_gini",
    y="clone_centrality_gini",
    data=gini,
    hue=gini.index,
    palette=dict(
        zip(adata.obs["sampleid"].cat.categories, adata.uns["sampleid_colors"])
    ),
)
p.set(ylim=(-0.1, 1), xlim=(-0.1, 1))
plt.legend(bbox_to_anchor=(1, 0.5), loc="center left", frameon=False)
p

Chao1 is an estimator based on abundance

In [ ]:
ddl.tl.clone_diversity(
    vdj, groupby="sample_id", method="chao1", return_table=True
)

For Shannon Entropy, we can calculate a normalized (inspired by [scirpy's function](https://icbi-lab.github.io/scirpy/generated/scirpy.tl.alpha_diversity.html?highlight=diversity#scirpy.tl.alpha_diversity)) and non-normalized value.

In [ ]:
ddl.tl.clone_diversity(
    vdj, groupby="sample_id", method="shannon", return_table=True
)

In [ ]:
ddl.tl.clone_diversity(
    vdj,
    groupby="sample_id",
    method="shannon",
    update_obs_meta=False,
    normalize=False,
)

That sums it up for now! Let me know if you have any ideas at [z.tuong@uq.edu.au] and I can try and see if i can implement it or we can work something out to collaborate on!

In [ ]:
vdj2.write_h5ddl("test.h5ddl", compression="gzip")